# Intro

The task consists of classifying images into those that contain St. George and those that don't.
I will train several pre-trained models on the given data.

# Data Exploration

Let's take a look at the images

In [ ]:
import pandas as pd
import requests
import itertools
import torch
import torch.nn as nn, torch.optim as optim
from torchvision import models, transforms, datasets
import albumentations as alb
import albumentations.augmentations.transforms as aat
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
df_george = pd.read_csv("./image_urls/georges.csv")
df_george.head()

In [ ]:
df_non_george = pd.read_csv("./image_urls/non_georges.csv")
df_non_george.head()

In [ ]:
george_links = set(itertools.chain(*pd.read_csv("./image_urls/georges.csv").values.tolist()))
non_george_links = set(itertools.chain(*pd.read_csv("./image_urls/non_georges.csv").values.tolist()))

Let's check if there are some bad photos, (photos both classified as george appearing and george not appearing).

In [ ]:
miss_labeled = george_links.intersection(non_george_links)
miss_labeled

We can notice there are few links in both classes. Let's ignore these links

In [ ]:
george_links.difference_update(miss_labeled)
non_george_links.difference_update(miss_labeled)

assert len(set(george_links).intersection(set(non_george_links))) == 0

In [ ]:
len(george_links), len(non_george_links)

In [ ]:
data = []
for link in george_links:
    data.append([link, 0])
for link in non_george_links:
    data.append([link, 1])
data = np.array(data)
data.shape

### Let's now save

In [ ]:
! mkdir data
! mkdir data/train
! mkdir data/test
! mkdir data/train/george
! mkdir data/train/non_george
! mkdir data/test/george
! mkdir data/test/non_george

In [ ]:
def save_image(path: str, url: str, idx: int) -> None:
    response = requests.get(url)
    if response.status_code == 200:
        image_data = response.content
        with open(f"{path}/img_{idx}.jpg", 'wb') as file:
            file.write(image_data)

In [ ]:
train_data, test_data = train_test_split(data, random_state=42)

In [ ]:
for idx, (link, label) in enumerate(train_data):
    if label == '0':
        save_image(path='data/train/george', url=link, idx=idx)
    elif label == '1':
        save_image(path='data/train/non_george', url=link, idx=idx)

In [ ]:
for idx, (link, label) in enumerate(test_data):
    if label == '0':
        save_image(path='data/test/george', url=link, idx=idx)
    elif label == '1':
        save_image(path='data/test/non_george', url=link, idx=idx)

# Loading and Augmenting Data

Let's load the data!

In [ ]:
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

In [ ]:
class AlbuWrapper:  # typing: ignore
    def __init__(self, atrans: alb.BasicTransform):
        self.atrans = atrans

    def __call__(self, img: Image.Image) -> Image.Image:
        return self.atrans(image=np.array(img))["image"]

In [ ]:
alb_transforms = alb.Compose(
    [
        alb.GaussNoise(),
        alb.OneOf(
            [alb.MotionBlur(p=0.2), alb.MedianBlur(blur_limit=3, p=0.1), alb.Blur(blur_limit=3, p=0.1)], p=0.2
        ),
        alb.OneOf([alb.OpticalDistortion(p=0.3), alb.GridDistortion(p=0.1), alb.PiecewiseAffine(p=0.3)], p=0.2),
        alb.OneOf([aat.CLAHE(clip_limit=2), alb.Sharpen(), alb.Emboss()], p=0.3),
        aat.HueSaturationValue(p=0.3),
        alb.HorizontalFlip(),
        aat.RGBShift(),
        aat.RandomBrightnessContrast(),
        aat.RandomGamma(),
        alb.CoarseDropout(2, 10, 10)
    ])

In [ ]:
train_transforms = transforms.Compose(
    [AlbuWrapper(alb_transforms), transform])
test_transforms = transform

In [ ]:
toPIL = transforms.ToPILImage()

In [ ]:
train_dataset = datasets.ImageFolder('./data/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.ImageFolder('./data/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
toPIL(next(iter(train_loader))[0][0][0])

# Training

Let's use a pretrained VGG16 model, and retrain it on our data.

In [ ]:
dtype = torch.float

device_name = 'cpu'

if torch.cuda.is_available(): # NVIDIA GPU
    device_name = 'cuda'
elif torch.backends.mps.is_available() and torch.backends.mps.is_built(): # Apple Silicon GPU
    device_name = 'mps'

device = torch.device(device_name)
device

In [ ]:
vgg_16_pretrained = models.vgg16(weights = models.VGG16_Weights.DEFAULT)
vgg_16_pretrained

Let's change the classifier to `output_features = 2`, as we're doing binary classification.
Also, we can notice that the `avgpool` layer has an output size of `7x7`. Let's change it to 1, so that it takes less time to train.

In [ ]:
vgg_16_pretrained.classifier = nn.Sequential(
    nn.Linear(in_features=512, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=2, bias=True)
)
vgg_16_pretrained.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
vgg_16_pretrained

Let's define our training and testing helpers.

In [ ]:
def train(model, train_loader, optimizer, loss_function):
    print('Training...')
    model.train()
    model = model.to(device)
    loss_function = loss_function.to(device)
    for batch_idx, (data, label) in tqdm(enumerate(train_loader)):
        data = data.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, test_loader, loss_function):
    print(f'Validating ...')
    model.eval()

    test_loss = 0
    num_correct = 0
    cnt = 0

    model = model.to(device=device)
    loss_function = loss_function.to(device=device)

    with torch.no_grad():
        for data, label in tqdm(test_loader):
            data = data.to(device=device)
            label = label.to(device=device)
            output = model(data)
            test_loss += loss_function(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            num_correct += pred.eq(label.view_as(pred)).sum().item()
            cnt += 1

    test_loss /= cnt

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

For training, we'll first freeze the first few layers of the feature extractor.
And later on, we'll unfreeze them and continue training.

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
vgg_16_pretrained = vgg_16_pretrained.to(device)

In [ ]:
for layer in vgg_16_pretrained.features[0:19]:
    for p in layer.parameters():
        p.requires_grad = False

optimizer = optim.Adam(params=vgg_16_pretrained.parameters(), lr=1e-4)

for epoch in range(0, 30):
    train(model=vgg_16_pretrained, train_loader=train_loader, optimizer=optimizer, loss_function=criterion)
    test(model=vgg_16_pretrained, test_loader=test_loader, loss_function=criterion)

We end up with $\approx$ 90% accuracy. Nice! Let's save the model at this state.

In [ ]:
! mkdir checkpoints

In [ ]:
state = {"model_state": vgg_16_pretrained.state_dict(),"optimizer_state": optimizer.state_dict(),"freeze":True}
torch.save(state['model_state'], "./checkpoints/conv_layers_frozen.pth")

In [ ]:
test(vgg_16_pretrained, test_loader=test_loader, loss_function=criterion)

In [104]:
model = vgg_16_pretrained
model.load_state_dict(torch.load('./checkpoints/conv_layers_frozen.pth'))

<All keys matched successfully>

# Classification metrics report

In [ ]:
true_labels = torch.tensor([]).to(device=device)
predicted_labels = torch.tensor([]).to(device=device)

with torch.no_grad():
    for data, label in tqdm(test_loader):
        data = data.to(device=device)
        label = label.to(device=device)
        output = model(data)
        pred = output.argmax(dim=1)
        true_labels = torch.cat([true_labels, label])
        predicted_labels = torch.cat([predicted_labels, pred])

 26%|██▌       | 6/23 [00:13<00:42,  2.50s/it]

In [ ]:
true_labels = np.array(true_labels.cpu())
predicted_labels = np.array(predicted_labels.cpu())

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true=true_labels, y_pred=predicted_labels))